In [8]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [9]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

highway_output_data = params['highway_output_data']
perf_measure_columns = params['final_columns']
periods = params['periods']
concept_id = params['concept_id']
summary_outputs = params['summary_dir']
filename_extension = params['filename_extension']

hwy_output = 0
if hwy_output:
    #am = pd.read_csv(_join(highway_output_data, 'Volumes and times (on links)_am.csv'), sep='\t')
    df_vmt = pd.DataFrame(columns=['Period', 'Value'])
    for prd in period:
        tod = pd.read_csv(_join(highway_output_data, f'Volumes and times (on links)_{prd}.csv'), sep='\t')    
        df_vmt = df_vmt.append({'Period': prd, 'Value': tod['VDT'].sum()}, ignore_index=True)
        
    df_vmt['Concept_ID'] = concept_id
    df_vmt['Metric_ID'] = 'A3.4'
    df_vmt['Metric_name'] = ''
    df_vmt['Submetric'] = 'A3.4.1'
    df_vmt['Description'] = 'VMT reduction (network)'
    df_vmt['Population'] = 'Whole Population'
    df_vmt['Geography'] = 'Regional'
    df_vmt['Zone_ID'] = ''
    df_vmt['Origin_zone'] = ''
    df_vmt['Dest_zone'] = ''
    df_vmt['Purpose'] = ''
    df_vmt['Units'] = 'vehicle miles travelled'
    df_vmt['Total_Increment'] = ''
    
    df_vmt.to_csv(_join(summary_dir, ''))

In [10]:
df = pd.read_csv(_join(highway_output_data))

df['am_vmt'] = (df['autovolAM'] + df['truck volumeAM'] + df['large truck linke volumeAM'])* df['distance']
df['pm_vmt'] = (df['autovolPM'] + df['truck volumePM'] + df['large truck linke volumePM'])* df['distance']
df['ea_vmt'] = (df['autovolEA'] + df['truck volumeEA'] + df['large truck linke volumeEA'])* df['distance']
df['md_vmt'] = (df['autovolMD'] + df['truck volumeMD'] + df['large truck linke volumeMD'])* df['distance']
df['ev_vmt'] = (df['autovolEV'] + df['truck volumeEV'] + df['large truck linke volumeEV'])* df['distance']

df_vmt = pd.DataFrame(perf_measure_columns)

df_temp = []
for per in periods:
    df_temp.append({'Period': per,
                    'Value': df[per+'_vmt'].sum()})
    
df_vmt = pd.DataFrame(df_temp)
all_row = {'Period' : 'All', 'Value': df_vmt['Value'].sum()}
df_vmt = df_vmt.append(all_row, ignore_index=True)

In [11]:
df_vmt

,Period,Value
0,am,5.164439e+07
1,md,6.300584e+07
2,pm,5.609260e+07
3,ev,3.492204e+07
4,ea,8.738900e+06
5,All,2.144038e+08


In [12]:
df_vmt['Concept_ID'] = concept_id
df_vmt['Metric_ID'] = 'A3.4'
df_vmt['Metric_name'] = ''
df_vmt['Submetric'] = 'A3.4.1'
df_vmt['Description'] = 'VMT reduction (network). It includes VMT by auto and trucks'
df_vmt['Population'] = 'Whole Population'
df_vmt['Geography'] = 'Regional'
df_vmt['Zone_ID'] = ''
df_vmt['Origin_zone'] = ''
df_vmt['Dest_zone'] = ''
df_vmt['Purpose'] = ''
df_vmt['Mode'] = ''
df_vmt['Income'] = ''
df_vmt['Units'] = 'vehicle miles travelled'
df_vmt['Total_Increment'] = ''

In [13]:
df_vmt = df_vmt[perf_measure_columns]
df_vmt.to_csv(_join(summary_outputs, 'A3.4.1' + '_weekday_VMT_' + concept_id + '_region' + filename_extension + '.csv'), index=False)

In [14]:
df_vmt['Value'] = df_vmt['Value'] * params['annual_auto_factor']
df_vmt['Submetric'] = 'A3.4.2'

df_vmt.to_csv(_join(summary_outputs, 'A3.4.2' + '_annual_VMT_' + concept_id + '_region' + filename_extension + '.csv'), index=False)